# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 12 2022 2:08PM,241682,19,ADV80004112,"AdvaGen Pharma, Ltd",Released
1,Jul 12 2022 2:01PM,241681,10,8265088,Eywa Pharma Inc.,Released
2,Jul 12 2022 2:01PM,241681,10,8265089,Eywa Pharma Inc.,Released
3,Jul 12 2022 2:01PM,241681,10,8265090,Eywa Pharma Inc.,Released
4,Jul 12 2022 2:00PM,241680,15,PNC367154A,"Person & Covey, Inc.",Released
5,Jul 12 2022 2:00PM,241680,15,PNC374149A,"Person & Covey, Inc.",Released
6,Jul 12 2022 2:00PM,241680,15,PNC374150A,"Person & Covey, Inc.",Released
7,Jul 12 2022 2:00PM,241680,15,PNC374151A,"Person & Covey, Inc.",Released
8,Jul 12 2022 2:00PM,241680,15,PNC374152A,"Person & Covey, Inc.",Released
9,Jul 12 2022 2:00PM,241680,15,PNC374153A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,241678,Released,12
42,241679,Released,1
43,241680,Released,6
44,241681,Released,3
45,241682,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241678,NaN,NaN,12.0
241679,NaN,NaN,1.0
241680,NaN,NaN,6.0
241681,NaN,NaN,3.0
241682,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241575,0.0,1.0,0.0
241593,3.0,0.0,1.0
241601,0.0,0.0,1.0
241605,0.0,38.0,19.0
241612,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241575,0,1,0
241593,3,0,1
241601,0,0,1
241605,0,38,19
241612,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241575,0,1,0
1,241593,3,0,1
2,241601,0,0,1
3,241605,0,38,19
4,241612,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241575,,1,
1,241593,3,,1
2,241601,,,1
3,241605,,38,19
4,241612,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 12 2022 2:08PM,241682,19,"AdvaGen Pharma, Ltd"
1,Jul 12 2022 2:01PM,241681,10,Eywa Pharma Inc.
4,Jul 12 2022 2:00PM,241680,15,"Person & Covey, Inc."
10,Jul 12 2022 1:47PM,241679,20,Alumier Labs Inc.
11,Jul 12 2022 1:42PM,241678,12,Hush Hush
23,Jul 12 2022 1:33PM,241677,10,Aerogen LTD.
24,Jul 12 2022 1:31PM,241676,10,"Citieffe, Inc."
26,Jul 12 2022 1:16PM,241673,20,Alumier Labs Inc.
43,Jul 12 2022 1:16PM,241674,22,"NBTY Global, Inc."
44,Jul 12 2022 1:11PM,241672,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 12 2022 2:08PM,241682,19,"AdvaGen Pharma, Ltd",,,1
1,Jul 12 2022 2:01PM,241681,10,Eywa Pharma Inc.,,,3
2,Jul 12 2022 2:00PM,241680,15,"Person & Covey, Inc.",,,6
3,Jul 12 2022 1:47PM,241679,20,Alumier Labs Inc.,,,1
4,Jul 12 2022 1:42PM,241678,12,Hush Hush,,,12
5,Jul 12 2022 1:33PM,241677,10,Aerogen LTD.,,,1
6,Jul 12 2022 1:31PM,241676,10,"Citieffe, Inc.",,,2
7,Jul 12 2022 1:16PM,241673,20,Alumier Labs Inc.,,,17
8,Jul 12 2022 1:16PM,241674,22,"NBTY Global, Inc.",,,1
9,Jul 12 2022 1:11PM,241672,20,"ACI Healthcare USA, Inc.",,,37


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 2:08PM,241682,19,"AdvaGen Pharma, Ltd",1,,
1,Jul 12 2022 2:01PM,241681,10,Eywa Pharma Inc.,3,,
2,Jul 12 2022 2:00PM,241680,15,"Person & Covey, Inc.",6,,
3,Jul 12 2022 1:47PM,241679,20,Alumier Labs Inc.,1,,
4,Jul 12 2022 1:42PM,241678,12,Hush Hush,12,,
5,Jul 12 2022 1:33PM,241677,10,Aerogen LTD.,1,,
6,Jul 12 2022 1:31PM,241676,10,"Citieffe, Inc.",2,,
7,Jul 12 2022 1:16PM,241673,20,Alumier Labs Inc.,17,,
8,Jul 12 2022 1:16PM,241674,22,"NBTY Global, Inc.",1,,
9,Jul 12 2022 1:11PM,241672,20,"ACI Healthcare USA, Inc.",37,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 2:08PM,241682,19,"AdvaGen Pharma, Ltd",1,,
1,Jul 12 2022 2:01PM,241681,10,Eywa Pharma Inc.,3,,
2,Jul 12 2022 2:00PM,241680,15,"Person & Covey, Inc.",6,,
3,Jul 12 2022 1:47PM,241679,20,Alumier Labs Inc.,1,,
4,Jul 12 2022 1:42PM,241678,12,Hush Hush,12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 2:08PM,241682,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Jul 12 2022 2:01PM,241681,10,Eywa Pharma Inc.,3.0,NaN,NaN
2,Jul 12 2022 2:00PM,241680,15,"Person & Covey, Inc.",6.0,NaN,NaN
3,Jul 12 2022 1:47PM,241679,20,Alumier Labs Inc.,1.0,NaN,NaN
4,Jul 12 2022 1:42PM,241678,12,Hush Hush,12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 2:08PM,241682,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Jul 12 2022 2:01PM,241681,10,Eywa Pharma Inc.,3.0,0.0,0.0
2,Jul 12 2022 2:00PM,241680,15,"Person & Covey, Inc.",6.0,0.0,0.0
3,Jul 12 2022 1:47PM,241679,20,Alumier Labs Inc.,1.0,0.0,0.0
4,Jul 12 2022 1:42PM,241678,12,Hush Hush,12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4349716,124.0,13.0,1.0
12,483323,12.0,1.0,0.0
15,483285,25.0,38.0,0.0
16,241601,1.0,0.0,0.0
18,241669,1.0,0.0,0.0
19,1449826,5.0,1.0,3.0
20,1449928,67.0,19.0,0.0
21,724899,3.0,0.0,0.0
22,483297,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4349716,124.0,13.0,1.0
1,12,483323,12.0,1.0,0.0
2,15,483285,25.0,38.0,0.0
3,16,241601,1.0,0.0,0.0
4,18,241669,1.0,0.0,0.0
5,19,1449826,5.0,1.0,3.0
6,20,1449928,67.0,19.0,0.0
7,21,724899,3.0,0.0,0.0
8,22,483297,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,124.0,13.0,1.0
1,12,12.0,1.0,0.0
2,15,25.0,38.0,0.0
3,16,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,5.0,1.0,3.0
6,20,67.0,19.0,0.0
7,21,3.0,0.0,0.0
8,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,124.0
1,12,Released,12.0
2,15,Released,25.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
Executing,13.0,1.0,38.0,0.0,0.0,1.0,19.0,0.0,1.0
Released,124.0,12.0,25.0,1.0,1.0,5.0,67.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,13.0,1.0,38.0,0.0,0.0,1.0,19.0,0.0,1.0
2,Released,124.0,12.0,25.0,1.0,1.0,5.0,67.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,13.0,1.0,38.0,0.0,0.0,1.0,19.0,0.0,1.0
2,Released,124.0,12.0,25.0,1.0,1.0,5.0,67.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()